## **FASE 3: DISEÑO, CREACIÓN DE BBDD INSERCIÓN DE DATOS**

- **Librerías**

In [1]:
# importar librerías

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Libreria Regex para buscar patrones
# -----------------------------------------------------------------------
import re

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Ajusta el ancho de la consola para evitar truncamientos
# -----------------------------------------------------------------------
pd.set_option("display.width", 1000) 

# Importar librería para la conexión con MySQL
# --------------------------------------------
import mysql.connector
from mysql.connector import errorcode

# Importar otras librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import csv

- **Lectura csv**

In [2]:
# Lectura del csv:

df_abc = pd.read_csv("Dataset_HR_final.csv", index_col = None)
df_abc.head()

,age,attrition,business_travel,daily_rate,department,distance_from_home,education,education_field,employee_count,employee_number,environment_satisfaction,gender,hourly_rate,job_involvement,job_level,job_role,job_satisfaction,marital_status,monthly_income,monthly_rate,num_companies_worked,over_time,percent_salary_hike,performance_rating,relationship_satisfaction,standard_hours,stock_option_level,total_working_years,training_times_last_year,work_life_balance,years_at_company,years_since_last_promotion,years_with_curr_manager,year_birth,salary,remote_work
0,51,No,Unknown,684.0,Research & Development,6,3,Other,1,162.0,1,Male,51.0,3,5,Research Director,3,Unknown,19537.0,6462,7,No,13,3,3,40,0,11.25,5,3.00,20,15,15,1972,246240.0,Yes
1,52,No,Unknown,699.0,Research & Development,1,4,Life Sciences,1,259.0,3,Male,65.0,2,5,Manager,3,Unknown,19999.0,5678,0,No,14,3,1,40,1,34.00,5,3.00,33,11,9,1971,251640.0,Yes
2,42,No,travel rarely,532.0,Research & Development,4,2,Technical Degree,1,319.0,3,Male,58.0,3,5,Manager,4,Married,19232.0,4933,1,No,11,3,4,40,0,22.00,3,2.77,22,11,15,1981,191520.0,Yes
3,47,No,travel rarely,359.0,Research & Development,2,4,Medical,1,1708.0,1,Female,82.0,3,4,Research Director,3,Married,17169.0,26703,3,No,19,3,2,40,2,11.25,2,2.77,20,5,6,1976,129240.0,No
4,48,No,Unknown,117.0,Research & Development,22,3,Medical,1,1900.0,4,Female,58.0,3,4,Manager,4,Unknown,17174.0,2437,3,No,11,3,2,40,1,11.25,3,3.00,22,4,7,1975,42120.0,Yes


- Data frame employees

In [3]:
# Tenemos 4 tablas y creamos 4 data frames

df_employees = df_abc[["employee_number", "age", "gender", "year_birth", "marital_status", "education", "education_field", "distance_from_home", "total_working_years", "num_companies_worked"]]
df_employees.head(2)

,employee_number,age,gender,year_birth,marital_status,education,education_field,distance_from_home,total_working_years,num_companies_worked
0,162.0,51,Male,1972,Unknown,3,Other,6,11.25,7
1,259.0,52,Male,1971,Unknown,4,Life Sciences,1,34.00,0


In [4]:
# convertimos en csv

df_employees.to_csv('Dataset_employees_HR.csv', index= False)

- Data frame job information

In [5]:
df_job_information = df_abc[["employee_number","employee_count","job_role",
    "department",
    "job_level" ,
    "training_times_last_year",
    "remote_work",
    "standard_hours",
    "over_time",
    "years_at_company",
    "years_since_last_promotion",
    "years_with_curr_manager",
    "attrition"]]

df_job_information.head(2)

,employee_number,employee_count,job_role,department,job_level,training_times_last_year,remote_work,standard_hours,over_time,years_at_company,years_since_last_promotion,years_with_curr_manager,attrition
0,162.0,1,Research Director,Research & Development,5,5,Yes,40,No,20,15,15,No
1,259.0,1,Manager,Research & Development,5,5,Yes,40,No,33,11,9,No


In [6]:
# convertimos en csv

df_job_information.to_csv('Dataset_job_information_HR.csv', index= False)

- Data frame job compesation

In [8]:
df_job_compensation = df_abc [["employee_number","daily_rate", "monthly_rate", "monthly_income", "hourly_rate","percent_salary_hike", "salary","stock_option_level"]]

df_job_compensation.head(2)

,employee_number,daily_rate,monthly_rate,monthly_income,hourly_rate,percent_salary_hike,salary,stock_option_level
0,162.0,684.0,6462,19537.0,51.0,13,246240.0,0
1,259.0,699.0,5678,19999.0,65.0,14,251640.0,1


In [11]:
# convertimos en csv

df_job_compensation.to_csv('Dataset_job_compensation_HR.csv', index= False)

- Data frame job satisfaction 

In [10]:
df_job_satisfaction  = df_abc [["employee_number", "job_involvement","performance_rating","environment_satisfaction" ,"relationship_satisfaction" ,"work_life_balance"]]

df_job_satisfaction.head(2)

,employee_number,job_involvement,performance_rating,environment_satisfaction,relationship_satisfaction,work_life_balance
0,162.0,3,3,1,3,3.0
1,259.0,2,3,3,1,3.0


In [12]:
# convertimos en csv

df_job_satisfaction.to_csv('Dataset_job_satisfaction_HR.csv', index= False)

 **A. CONEXIÓN CON SQL**

In [13]:
# Conectar a SQL
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')
# Crear cursor
mycursor = cnx.cursor()

# Crear base de datos
try:
    mycursor.execute("CREATE DATABASE BBDD_ABC_Corporation")
    print(mycursor)
except mysql.connector.Error as err: # verificar si hay errores
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

MySQLCursor: CREATE DATABASE BBDD_ABC_Corporation


**B. UBICACIÓN BBDD "BBDD_ABC_Corporation" Y CREACIÓN DE TABLAS CON SUS COLUMNAS**

- **B.1.** TABLA 1: employees

In [14]:
# Seleccionar la base de datos

try:
    mycursor.execute("USE BBDD_ABC_Corporation")
    print("Base de datos seleccionada correctamente.")
except mysql.connector.Error as err:
    print("Error al seleccionar la base de datos.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)
    
# Crear tabla 1: "employees"

try:
    mycursor.execute("""
  CREATE TABLE employees (
    employee_number INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    age INT,
    gender VARCHAR(10),
    year_birth YEAR,
    marital_status VARCHAR(20),
    education INT,
    education_field VARCHAR(50),
    distance_from_home INT,
    total_working_years INT,                 
    num_companies_worked INT
)  
""")

    # Verficar la creación de tabla 1: "employees"
    
    print("Tabla 'employees' creada exitosamente.")
except mysql.connector.Error as err:
    print("Se produjo un error al crear la tabla 'employees'.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)

Base de datos seleccionada correctamente.


Tabla 'employees' creada exitosamente.


- **B.2.** TABLA 2: job_information

In [19]:

# Seleccionar la base de datos

try:
    mycursor.execute("USE BBDD_ABC_Corporation")
    print("Base de datos seleccionada correctamente.")
except mysql.connector.Error as err:
    print("Error al seleccionar la base de datos.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)
    
# Crear tabla 2: "job_information"

try:
    mycursor.execute("""
    CREATE TABLE job_information (
    employee_number INT PRIMARY KEY,
    employee_count INT,
    job_role VARCHAR(50),
    department VARCHAR(50),
    job_level INT,
    training_times_last_year INT,
    remote_work VARCHAR(10),
    standard_hours INT,
    over_time VARCHAR(10),
    years_at_company INT,
    years_since_last_promotion INT,
    years_with_curr_manager INT,
    attrition VARCHAR(10),
    FOREIGN KEY (employee_number) REFERENCES employees(employee_number) ON DELETE CASCADE ON UPDATE CASCADE
) 
""")
    
    # Verficar la creación de tabla 2: "job_information"
    
    print("Tabla 'job_information' creada exitosamente.")
except mysql.connector.Error as err:
    print("Se produjo un error al crear la tabla 'job_information'.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)

Base de datos seleccionada correctamente.
Tabla 'job_information' creada exitosamente.


- **B.3.** TABLA 3: job_compensation

In [5]:

# Seleccionar la base de datos

try:
    mycursor.execute("USE BBDD_ABC_Corporation")
    print("Base de datos seleccionada correctamente.")
except mysql.connector.Error as err:
    print("Error al seleccionar la base de datos.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)
    
# Crear tabla 3: "job_compensation"

try:
    mycursor.execute("""
    CREATE TABLE job_compensation (
    employee_number INT PRIMARY KEY,
    daily_rate FLOAT,
    monthly_rate FLOAT,
    monthly_income FLOAT,
    hourly_rate FLOAT,
    percent_salary_hike INT,
    salary FLOAT,
    stock_option_level INT,                         
    FOREIGN KEY (employee_number) REFERENCES employees(employee_number) ON DELETE CASCADE ON UPDATE CASCADE
)
""")
    
    # Verficar la creación de tabla 3: "job_compensation"
    
    print("Tabla 'job_compensation' creada exitosamente.")
except mysql.connector.Error as err:
    print("Se produjo un error al crear la tabla 'job_compensation'.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)

Base de datos seleccionada correctamente.
Se produjo un error al crear la tabla 'job_compensation'.
Código de Error: 1050
SQLSTATE: 42S01
Mensaje: Table 'job_compensation' already exists


- **B.4.** TABLA 4: job_satisfaction

In [6]:

# Seleccionar la base de datos

try:
    mycursor.execute("USE BBDD_ABC_Corporation")
    print("Base de datos seleccionada correctamente.")
except mysql.connector.Error as err:
    print("Error al seleccionar la base de datos.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)
    
# Crear tabla 4: "job_satisfaction"

try:
    mycursor.execute("""
    CREATE TABLE job_satisfaction (
    employee_number INT PRIMARY KEY, 
    job_involvement INT,
    performance_rating INT,        
    environment_satisfaction INT,
    relationship_satisfaction INT,
    work_life_balance INT,            
    FOREIGN KEY (employee_number) REFERENCES employees(employee_number)ON DELETE CASCADE ON UPDATE CASCADE
)
""")
    
    # Verficar la creación de tabla 4: "job_satisfaction"
    
    print("Tabla 'job_satisfaction' creada exitosamente.")
except mysql.connector.Error as err:
    print("Se produjo un error al crear la tabla 'job_satisfaction'.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)

Base de datos seleccionada correctamente.
Se produjo un error al crear la tabla 'job_satisfaction'.
Código de Error: 1050
SQLSTATE: 42S01
Mensaje: Table 'job_satisfaction' already exists


**C. INSERCIÓN DE DATOS EN LAS TABLAS**

- **C.1.** TABLA 1: employees

In [16]:
# Insertar registros dentro de la tabla 1: "employees"

sql = "INSERT INTO employees (employee_number, age, gender, year_birth, marital_status, education, education_field, distance_from_home, total_working_years, num_companies_worked) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

val = df_employees[['employee_number', 'age', 'gender', 'year_birth', 'marital_status', 'education', 'education_field', 'distance_from_home', 'total_working_years', 'num_companies_worked']].values.tolist()

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registros insertados.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1080 registros insertados.


- **C.2.** TABLA 2: job_information

In [20]:
sql = "INSERT INTO job_information (employee_number, employee_count, job_role, department, job_level, training_times_last_year, remote_work, standard_hours, over_time, years_at_company, years_since_last_promotion, years_with_curr_manager, attrition) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

val = df_job_information[["employee_number","employee_count","job_role",
    "department",
    "job_level" ,
    "training_times_last_year",
    "remote_work",
    "standard_hours",
    "over_time",
    "years_at_company",
    "years_since_last_promotion",
    "years_with_curr_manager",
    "attrition"]].values.tolist()


try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registros insertados.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1080 registros insertados.


- **C.3.** TABLA 3: job_compensation

In [ ]:
sql = "INSERT INTO job_compensation(employee_number,daily_rate, monthly_rate, monthly_income, hourly_rate, percent_salary_hike, salary,stock_option_level) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"

val = df_job_compensation[["employee_number","daily_rate", "monthly_rate", "monthly_income", "hourly_rate","percent_salary_hike", "salary","stock_option_level"]].values.tolist()


try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registros insertados.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1080 registros insertados.


- **C.4.** TABLA 4: job_satisfaction

In [ ]:
sql = "INSERT INTO job_satisfaction(employee_number, job_involvement,performance_rating,environment_satisfaction ,relationship_satisfaction ,work_life_balance) VALUES (%s,%s,%s,%s,%s,%s)"

val = df_job_satisfaction [["employee_number", "job_involvement","performance_rating","environment_satisfaction" ,"relationship_satisfaction" ,"work_life_balance"]].values.tolist()


try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registros insertados.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1080 registros insertados.


- **D. CERRAR CONEXIÓN CON SQL**

In [ ]:
# Cerrar el cursor y la conexión a la base de datos
try:
    # Si el cursor está abierto, cerrarlo
    if mycursor:
        mycursor.close()
    
    # Si la conexión está abierta, cerrarla
    if cnx:
        cnx.close()
    
    print("Conexión cerrada correctamente.")
except mysql.connector.Error as err:
    print("Error al cerrar la conexión:", err)